# Notebook to run docker building and running

## First step: build our own docker image

In [9]:
%cd ~/SageMaker/gnn_fraud_detection_dgl/

/home/ec2-user/SageMaker/docker_test_folder


In [10]:
! pwd

/home/ec2-user/SageMaker/docker_test_folder


In [16]:
! aws ecr get-login-password --region cn-north-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [17]:
! docker build -t pytorch-extended-container-test .

Sending build context to Docker daemon   59.9kB
Step 1/6 : FROM 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/pytorch-training:1.4.0-cpu-py36-ubuntu16.04
 ---> 9aae29fca0da
Step 2/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 43e63d19b750
Step 3/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> aba791b432d5
Step 4/6 : COPY * /opt/ml/code/
 ---> Using cache
 ---> f2d7fc170fd9
Step 5/6 : ENV SAGEMAKER_PROGRAM fd_sl_train_entry_point.py
 ---> Using cache
 ---> ce2f46fe8ab2
Step 6/6 : RUN pip install dgl==0.4.2
 ---> Using cache
 ---> 8d3fbb8eb7e4
Successfully built 8d3fbb8eb7e4
Successfully tagged pytorch-extended-container-test:latest


## Step 2: Test this docker image

In [18]:
from sagemaker import get_execution_role, Session

role = get_execution_role()
sess = Session()

print(role)
print(sess)

arn:aws-cn:iam::510768346845:role/service-role/AmazonSageMaker-ExecutionRole-20200526T111603


In [19]:
# Replace with an S3 location or local path to point to customers' own dataset
data_location = 'fraud-detection-solution'
raw_data_folder = 'raw_data'
processed_data_folder = 'processed_data'
model_output_folder = 'model_output'

processed_data = 's3://{}/{}'.format(data_location, processed_data_folder)
output_path = 's3://{}/{}'.format(data_location, model_output_folder)

print(processed_data)
print(output_path)

from os import path
from sagemaker.s3 import S3Downloader
processed_files = S3Downloader.list(processed_data)
print("===== Processed Files =====")
print('\n'.join(processed_files))

s3://fraud-detection-solution/processed_data
s3://fraud-detection-solution/model_output
===== Processed Files =====
s3://fraud-detection-solution/processed_data
s3://fraud-detection-solution/processed_data/features.csv
s3://fraud-detection-solution/processed_data/relation_DeviceInfo_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_DeviceType_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_P_emaildomain_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_ProductCD_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_R_emaildomain_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_TransactionID_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_addr1_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_addr2_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_card1_edgelist.csv
s3://fraud-detection-solution/processed_data/relation_card2_edgelist.csv
s3://fraud-de

In [7]:
edges = ",".join(map(lambda x: x.split("/")[-1], [file for file in processed_files if "relation" in file]))
params = {'nodes' : 'features.csv',
          'edges': 'relation*',
          'labels': 'tags.csv',
          'embedding-size': 64,
          'n-layers': 2,
          'n-epochs': 10,
          'optimizer': 'adam',
          'lr': 1e-2
        }

In [ ]:
from sagemaker.estimator import Estimator
from time import strftime, gmtime

estimator = Estimator(image_uri='pytorch-extended-container-test',
                      role=role,
                      train_instance_count=1,
                      train_instance_type='local',
                      hyperparameters=params,
                      output_path=output_path)

training_job_name = "{}-{}".format('GNN_FD_SL_DGL_Train', strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print(training_job_name)

estimator.fit({'train': processed_data}, job_name=training_job_name)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


JamesTrain01-2021-01-22-07-27-59
Creating tmp03mj5pi2_algo-1-v9h10_1 ... 
Attaching to tmp03mj5pi2_algo-1-v9h10_12mdone
algo-1-v9h10_1  | 2021-01-22 07:28:25,336 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-v9h10_1  | 2021-01-22 07:28:25,337 sagemaker-containers INFO     Failed to parse hyperparameter nodes value features.csv to Json.
algo-1-v9h10_1  | Returning the value itself
algo-1-v9h10_1  | 2021-01-22 07:28:25,337 sagemaker-containers INFO     Failed to parse hyperparameter edges value relation* to Json.
algo-1-v9h10_1  | Returning the value itself
algo-1-v9h10_1  | 2021-01-22 07:28:25,337 sagemaker-containers INFO     Failed to parse hyperparameter labels value tags.csv to Json.
algo-1-v9h10_1  | Returning the value itself
algo-1-v9h10_1  | 2021-01-22 07:28:25,338 sagemaker-containers INFO     Failed to parse hyperparameter model value rgcn to Json.
algo-1-v9h10_1  | Returning the value itself
algo-1-v9h10_1  | 2021-01-22 07:28:25,

In [23]:
!pwd

/home/ec2-user/SageMaker/docker_test_folder
